# Calculate Portfolio Dividends with OPENBB

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from openbb_terminal.sdk import openbb
#  Get information from dividends
#  openbb.stocks.fa.divs("ticker")
openbb.login(token='YOUR_TOKEN_HERE')

# Get information from stocks from OpenBB
ticks_with_divs = ['MSFT','AAPL','VZ','AMZN']

df = pd.DataFrame()

for tick in ticks_with_divs:
    div_tick = openbb.stocks.fa.divs(tick)
    div_tick['Ticker'] = tick
    div_tick = div_tick[div_tick.index>='2021-01-01']
    df = pd.concat([df, div_tick], axis = 0)

df = df[df.index >= '2021-01-01']
df['Year'] = df.index.year.astype('category')


df2 = df.groupby(['Ticker','Year']).agg({'Dividends':'sum'}).reset_index()
df2['Year']= df2['Year'].astype('category') # To use it in the bar plot


# Create some Bar Plots with Plotly

In [35]:

import plotly.express as px
fig = px.bar(df2, x="Ticker", y="Dividends", color = 'Year', title="Dividends per Year", width=1000,height=600,text_auto=True)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [36]:
df['Month'] = df.index.month

df['Month']= df['Month'].astype('category')
fig = px.bar(df, x="Month", y=df["Dividends"], color = 'Ticker',
             title="Dividends per Month", width=1000,height=600,text_auto=True)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [ ]:
# Calculate dividend returns according the number of stocks
taxes = 0.34 # This figure contains the total taxes 19% in Spain + 15% paid in the US 
n_stocks = {'AMZN':100, 'AAPL':100, 'MSFT':100, 'VZ':100}

USD_EUR = openbb.forex.load(from_symbol="USD", to_symbol="EUR", start_date="2023-01-01", end_date=None)
#USD_EUR.iloc[-1,3] # Close

#print(1/USD_EUR.iloc[-1,3])
patterns = [df['Ticker'] == 'AMZN',
            df['Ticker'] == 'AAPL',
            df['Ticker'] == 'MSFT',
            df['Ticker'] == 'VZ',

            ]

df = (df
 .assign(Number_of_positions = np.select(patterns,  n_stocks.values()),
         Div_ret_after_tax_in_EUR= lambda df_: df_['Number_of_positions']*df_['Dividends']*(1-taxes)*USD_EUR.iloc[-1,3]
         )
 .sort_values(by = ['Month', 'Ticker'], ascending=False)
)



In [47]:
fig = px.bar(df, x="Month", y="Div_ret_after_tax_in_EUR", color = 'Ticker',
             title="Div returns after tax in EUR per Month since 2021", width=1000,height=600,text_auto=True)
fig.update_layout(barmode='stack', xaxis_tickangle=-45)
fig.show()

In [48]:

import plotly.express as px
fig = px.bar(df, x="Ticker", y="Div_ret_after_tax_in_EUR", color = 'Year', 
             title="Dividends per Year for 100 stocks each", width=1000,height=600,text_auto=True)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()